#Regressão Múltipla usando uma base Video games

In [ ]:
#Importar dados essenciais
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive/')

dataset = pd.read_csv('/content/drive/My Drive/CursoIA/MulticlassRegressorVideoGames/games.csv')
dataset

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Importando libs do keras para criação da rede neural de regressão
from keras.layers  import Dense, Dropout, Activation, Input
from keras. models import Model

#Pré-processamento no dataset

In [ ]:
#Apagando dados não necessários para o dataset
dataset = dataset.drop('Other_Sales', axis = 1)
dataset = dataset.drop('Global_Sales', axis = 1)
dataset = dataset.drop('Developer', axis = 1)

In [ ]:
#Apagando dados NAN
dataset = dataset.dropna(axis=0)
print(f'Count lines: {len(dataset)}')


Count lines: 6825


In [ ]:
#Apagando dados para evitar erros de correlação
dataset = dataset.loc[dataset['NA_Sales'] > 1 ]
dataset = dataset.loc[dataset['EU_Sales'] > 1 ]
nome_jogos = dataset.Name
dataset = dataset.drop('Name', axis = 1)

In [ ]:
previsores = dataset.iloc[:,[0,1,2,3,7,8,9,10,1]].values
venda_na = dataset.iloc[:,4].values
venda_eu = dataset.iloc[:,5].values
venda_jp = dataset.iloc[:,6].values

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelEncoder = LabelEncoder()
previsores[:,0] = labelEncoder.fit_transform(previsores[:,0])
previsores[:,2] = labelEncoder.fit_transform(previsores[:,2])
previsores[:,3] = labelEncoder.fit_transform(previsores[:,3])
previsores[:,8] = labelEncoder.fit_transform(previsores[:,8])


columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0,2,3,8])],     remainder='passthrough')
previsores=columnTransformer.fit_transform(previsores).toarray()

#Rede Neural

In [ ]:
camada_entrada = Input(shape=(78,))
camada_oculta1 = Dense(units = 41, activation = 'sigmoid')(camada_entrada)
camada_oculta2 = Dense(units = 41, activation = 'sigmoid')(camada_oculta1)
camada_saida1 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida2 = Dense(units = 1, activation = 'linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation = 'linear')(camada_oculta2)


regressor = Model(inputs = camada_entrada,
                  outputs = [camada_saida1, camada_saida2, camada_saida3])

regressor.compile(optimizer = 'adam',
                  loss = 'mse')

In [ ]:
#Treinamento
regressor.fit(previsores,
              [venda_na, venda_eu, venda_jp],
              epochs = 5000,
              batch_size = 100)

Streaming output truncated to the last 5000 lines.
3/3 [==============================] - 0s 4ms/step - loss: 2.9203 - dense_7_loss: 1.3229 - dense_8_loss: 1.0002 - dense_9_loss: 0.5972
Epoch 2502/5000
3/3 [==============================] - 0s 4ms/step - loss: 2.9344 - dense_7_loss: 1.3798 - dense_8_loss: 0.9852 - dense_9_loss: 0.5695
Epoch 2503/5000
3/3 [==============================] - 0s 4ms/step - loss: 3.1791 - dense_7_loss: 1.5127 - dense_8_loss: 1.0714 - dense_9_loss: 0.5950
Epoch 2504/5000
3/3 [==============================] - 0s 5ms/step - loss: 3.0626 - dense_7_loss: 1.4434 - dense_8_loss: 1.0603 - dense_9_loss: 0.5588
Epoch 2505/5000
3/3 [==============================] - 0s 4ms/step - loss: 3.1160 - dense_7_loss: 1.4942 - dense_8_loss: 1.0322 - dense_9_loss: 0.5896
Epoch 2506/5000
3/3 [==============================] - 0s 4ms/step - loss: 3.0027 - dense_7_loss: 1.3256 - dense_8_loss: 1.0526 - dense_9_loss: 0.6244
Epoch 2507/5000
3/3 [==============================] - 0s 4

#Previsão

In [ ]:
previsa_na , previsao_eu, previsao_jp = regressor.predict(previsores)
print(previsa_na.mean() , previsao_eu.mean(), previsao_jp.mean() )
print(previsa_na.mean() , previsao_eu.mean(), previsao_jp.mean() )

9/9 [==============================] - 0s 2ms/step
3.4809694 2.4224634 0.5613331
